# Collateral Repo

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) and the Refinitiv Data Libraries.

##### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.
 
 You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 


## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import repo
from refinitiv.data.content.ipa.financial_contracts import bond
import pandas as pd

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x1061a1af0 {name='workspace'}>

### Repo interest & repurchase price

This sample computes the repurchase price and the interest due for a collateral repo. In this example the repo:
- repo starts on 2021-01-21
- repo ends on 2021-01-22
- interest paid is 3.03%
- the daycouont basis is Dcb_Actual_360
- the underlying bonds is a Indonesian government bonds (FR0077)
- the bpnd notional is IDR300,000,000,000
- the bond dirty price is 109.330619
- a 10% haircut is applied

in addition the the parameters above the following 'fields' are requested:
- the InitialMarginPercent
- the RepoInterestAmountInDealCcy
- the RepurchasePrice that is computed from the purchase price and the repo interest
- UnderlyingAtStartObject that represents the state at the beginning of the repo of the bond used as collateral
- the error information (ErrorCode,ErrorMessage)


In [3]:
response = repo.Definition(

    start_date="2021-01-21",
    end_date="2021-01-22",
    repo_rate_percent=3.03,
    day_count_basis="Dcb_Actual_360",
    is_coupon_exchanged=False,

    underlying_instruments=[
        repo.UnderlyingContract(
            instrument_type="Bond",
            instrument_definition=bond.Definition(
                instrument_code="FR0077",
                notional_amount=300000000000
            ),
            pricing_parameters={
                "PricingParametersAtStart": {"dirtyPrice": 109.330619},
                "repoParameters": {"haircutRatePercent": 10}
            },
        )
    ],

    pricing_parameters=repo.PricingParameters(valuation_date="2021-01-21"),

    fields=["InstrumentTag",
            "StartDate",
            "EndDate",
            "SettlementConvention",
            "DayCountBasis"
            "RepoRatePercent",
            "HaircutRatePercent",
            "InitialMarginPercent",
            "PurchasePrice",
            "RepurchasePrice",
            "RepoInterestAmountInDealCcy",
            "UnderlyingAtStartObject",
            "ErrorCode",
            "ErrorMessage"],
).get_data()

response.data.df

,InstrumentTag,StartDate,EndDate,SettlementConvention,DayCountBasisRepoRatePercent,HaircutRatePercent,InitialMarginPercent,PurchasePrice,RepurchasePrice,RepoInterestAmountInDealCcy,UnderlyingAtStartObject,ErrorCode,ErrorMessage
0,<NA>,2021-01-21,2021-01-22,2WD,<NA>,10,111.111111,295192671300,295217516683.16803,24845383.167725,"{'endDate': {'value': '2024-05-15T00:00:00Z', ...",,


### Displaying the state of the bond posted as collateral at the start of the repo

In [4]:

pd.DataFrame(response.data.df["UnderlyingAtStartObject"][0])

,endDate,accrued,cleanPrice,dirtyPrice,notionalAmount,accruedDays,couponRatePercent,marketValueInDealCcy,yieldPercent
value,2024-05-15T00:00:00Z,1.5038,107.826819,109.330619,300000000000.0,67,8.125,327991857000.0,5.5054
status,Data,Computed,Computed,User,User,Computed,Data,Computed,Computed


In [5]:
rd.close_session()